<a href="https://colab.research.google.com/github/sonhj110/SESAC_lecture/blob/master/231018_%ED%86%A0%ED%94%BD%EB%AA%A8%EB%8D%B8%EB%A7%81_%EB%A6%AC%EB%B7%B0%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습2 - 리뷰 데이터로 토픽 모델링

pyLDAvis 를 사용해서 토픽모델링을 진행해보고,
다음 데이터 중 부정적인 리뷰를 몇개의 토픽으로 분리하는 것이 가장적절한지 적절한 토픽갯수를 찾아보세요.

In [1]:
# 경고메세지 끄기
import warnings
warnings.filterwarnings(action='ignore')

# 다시 키기
# warnings.filterwarnings(action='default')

## CSV 파일 가져오기

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [144]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/나의 실습파일/review.csv',
                 sep='\t')  # 탭으로 구분

# label; : 0 - 부정, 1 - 긍정

# df = df[0:1000]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [145]:
neg = df[df['label'] == 0]
neg_doc = neg['document']
neg_doc

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0                                     아 더빙.. 진짜 짜증나네요 목소리
2                                       너무재밓었다그래서보는것을추천한다
3                           교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
5           막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.
6                                   원작의 긴장감을 제대로 살려내지못했다.
                              ...                        
9992                                        5분은 재밌었으니 별2개
9993    1점은 아니고 한 6점 정도 주고 싶은데 평점이 너무 높아서 1점 줌. 영화가 의외...
9995    곰티비로 무료로 봤기때문에 5점주려고했는데 1 한국 공포영화의 특징인 깜놀시키려 하...
9996    이딴걸드라마라고썼냐 수습할수없으면걍친자녀아니면되고 간단하네 얼굴을바꿨으면 결말이라도...
9999                                        그냥보다나옴 노답 핵노잼
Name: document, Length: 5021, dtype: object

In [146]:
# 댓글만 리스트 형태로 저장
n_comments = neg_doc.values

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [147]:
print(n_comments[0:10])

['아 더빙.. 진짜 짜증나네요 목소리' '너무재밓었다그래서보는것을추천한다' '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'
 '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.' '원작의 긴장감을 제대로 살려내지못했다.'
 '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네'
 '울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해'
 '취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지'
 '굿바이 레닌 표절인것은 이해하는데 왜 뒤로 갈수록 재미없어지냐'
 '재미없다 지루하고. 같은 음식 영화인데도 바베트의 만찬하고 넘 차이남....바베트의 만찬은 이야기도 있고 음식 보는재미도 있는데 ; 이건 볼게없다 음식도 별로 안나오고, 핀란드 풍경이라도 구경할랫는데 그것도 별로 안나옴 ㅡㅡ']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## tokenizer 함수화

In [148]:
# !pip install konlpy

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [149]:
from konlpy.tag import Okt

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [204]:
def tokenizer(text) :
  okt = Okt()
  tags = okt.pos(text)

  stopPos = ['Josa', 'Punctuation', 'Suffix', 'Foreign', 'KoreanParticle', 'Number', 'Exclamation', 'URL', 'Adverb']
  stopWord = ['영화', '않다', '했다', '이건', '너무', '하는', '진짜',  '정말',
              '그냥', '전혀', '건가', '건지', '이런', '하나', '입니다', '보기', '정도',
              '없다', '완전', '솔직히', '절대', '없음', '없어요', 'OOO',
              '되는', '없고', 'OO', '였다', '없는', '보다', '같은', '보고', '같다', '보지']


  words = []
  for tag in tags :
    if tag[1] not in stopPos:
      if tag[0] not in stopWord and len(tag[0]) != 1 :
        words.append(tag[0])

  return words


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# gensim으로 tfidf 구하기

In [215]:
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [216]:
tknz_neg = []

for doc in n_comments :
  tknz_neg.append(tokenizer(doc))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [217]:
id2word = corpora.Dictionary(tknz_neg)
print(id2word)

Dictionary<12476 unique tokens: ['더빙', '목소리', '짜증나네요', '다그', '래서']...>


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [218]:
id2word

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [219]:
len(id2word)  # 고유 토큰수

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


12476

In [220]:
corpusTDM = []

for doc in tknz_neg :
  # print(doc)
  result = id2word.doc2bow(doc)
  # print(result)  # (인덱스, 등장횟수)
  corpusTDM.append(result)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [221]:
print(corpusTDM)  # 문서수

[[(0, 1), (1, 1), (2, 1)], [(3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)], [(9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)], [(15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(22, 1), (23, 1), (24, 1), (25, 1)], [(16, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)], [(38, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1)], [(47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 2), (55, 1), (56, 1)], [(12, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1)], [(11, 1), (12, 1), (28, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 2), (69, 2), (70, 1), (71, 3), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1)], [(80, 1), (81, 1), (82, 1), (83, 1)], [(84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1)], [(90, 1), (91, 1)], [(92, 1), (93, 1), (94, 1), (95, 1), (96, 1)], [(97, 1), (98, 2)], [(38, 1), (99, 1), (100, 1), (101,

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [222]:
tfidf = TfidfModel(corpusTDM)
corpusTfidf = tfidf[corpusTDM]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [223]:
n = 3
lda = LdaModel(corpus = corpusTfidf,
               id2word = id2word,
               num_topics = n,
               random_state = 100)

for t in lda.print_topics() :
  print(t[0], ":", t[1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0 : 0.004*"아깝다" + 0.003*"재미" + 0.002*"스토리" + 0.002*"졸작" + 0.001*"점수" + 0.001*"무슨" + 0.001*"감독" + 0.001*"작품" + 0.001*"재미없음" + 0.001*"여자"
1 : 0.006*"쓰레기" + 0.004*"별로" + 0.003*"시간" + 0.003*"평점" + 0.003*"재미없다" + 0.002*"내용" + 0.002*"스토리" + 0.002*"실망" + 0.002*"최악" + 0.002*"생각"
2 : 0.003*"최악" + 0.002*"감독" + 0.002*"결말" + 0.002*"연기" + 0.002*"평점" + 0.002*"스토리" + 0.002*"뻔한" + 0.002*"노잼" + 0.002*"처음" + 0.001*"만든"


## pyLDAvis로 시각화

In [224]:
# !pip install pyLDAvis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [225]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [226]:
# !pip install loky

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [227]:
import sys
import time
import traceback
from loky import set_loky_pickler
from loky import get_reusable_executor
from loky import wrap_non_picklable_objects

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [228]:
# !pip install pandas -U

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [229]:
# !pip install pandas==1.5.1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [230]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()  # 주피터 노트북 환경에서 사용하겠다는 선언
vis = gensimvis.prepare(lda, corpusTfidf, id2word)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
